In [1]:
!kaggle datasets download -d alfrandom/protein-secondary-structure
!unzip protein-secondary-structure.zip
!rm -rf protein-secondary-structure.zip
!mkdir dataset
!mv 2018-06-06-pdb-intersect-pisces.csv dataset/2018-06-06-pdb-intersect-pisces.csv
!mv 2018-06-06-ss.cleaned.csv dataset/2018-06-06-ss.cleaned.csv

100%|██████████████████████████████████████| 38.8M/38.8M [00:06<00:00, 6.22MB/s]
100%|██████████████████████████████████████| 38.8M/38.8M [00:06<00:00, 6.04MB/s]
Archive:  protein-secondary-structure.zip
  inflating: 2018-06-06-pdb-intersect-pisces.csv  
  inflating: 2018-06-06-ss.cleaned.csv  
